In [1]:
import re
import numpy as np
import pandas as pd
import preprocess_kgptalkie as ps

In [2]:
# !pip install joblib --upgrade

In [3]:
df = pd.read_csv("dataset.csv")

In [4]:
df.head()

,author,date,title,selftext,text,label
0,NightmareWarrior365,2022-02-03 16:23:16+00:00,is anybodys symptoms calm on certain days,some days it is terrible some days it is mild ...,is anybodys symptoms calm on certain days some...,1
1,Valiquette_sjm,2022-02-03 14:52:44+00:00,what is voodoo,i suffer from thought broadcasting i believe m...,what is voodoo i suffer from thought broadcast...,1
2,blueowl89,2022-02-03 13:57:24+00:00,are you a morning person or a night owl,i am 100 a night owl myself i think best at ni...,are you a morning person or a night owl i am 1...,1
3,Sz728,2022-02-03 11:20:04+00:00,ugh cant sleep,sorry i always vent here stuff that nobody wan...,ugh cant sleep sorry i always vent here stuff ...,1
4,ToughtSpiritNow,2022-02-03 10:45:51+00:00,that is had it,constant battle everything needs to be done i ...,that is had it constant battle everything need...,1


In [5]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [7]:
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,3), analyzer='char')

In [8]:
X = tfidf.fit_transform(df['text'])
y = df['label']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [10]:
model = LinearSVC()
model.fit(x_train,y_train)

LinearSVC()

In [11]:
y_pred = model.predict(x_test)

In [12]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      1143
           1       0.92      0.93      0.93      1137

    accuracy                           0.93      2280
   macro avg       0.93      0.93      0.93      2280
weighted avg       0.93      0.93      0.93      2280



In [13]:
t = "I would hear something that sound like a plane engine or like a really… you know… a really far off motor. It never went away entirely. It’s gone a lot more in the past couple of months since Christmas. It just sounds like that… it sounds like a little flame or a cellular… a digital motor"
u = "Adults tell me(m15) some funny life stories of yours, give me life advice, all that stuff. I’m also trying to stop procrastinating so any advice on how to fix that would be great as well."

In [14]:
print(type(t))

<class 'str'>


In [15]:
def predict_tendency(model, x):
    x = get_clean(x)
    vec = tfidf.transform([x])
    print(vec.shape)
    if model.predict(vec)[0] == 1:
        print("The model predicts the statement to be: Schizophrenic")
    else:
        print("The model predicts the statement to be: Not Schizophrenic")

In [16]:
print(t)
print("-----------------------------------------------")
print(u)

I would hear something that sound like a plane engine or like a really… you know… a really far off motor. It never went away entirely. It’s gone a lot more in the past couple of months since Christmas. It just sounds like that… it sounds like a little flame or a cellular… a digital motor
-----------------------------------------------
Adults tell me(m15) some funny life stories of yours, give me life advice, all that stuff. I’m also trying to stop procrastinating so any advice on how to fix that would be great as well.


In [19]:
predict_tendency(model, t)
predict_tendency(model, u)

(1, 15947)
The model predicts the statement to be: Schizophrenic
(1, 15947)
The model predicts the statement to be: Not Schizophrenic


In [18]:
accuracy = model.score(x_test,y_test)
print("The accuracy of the model is: ",accuracy*100,"%.")

The accuracy of the model is:  92.71929824561404 %.


In [49]:
#Saving Model
import joblib
# pkl_file = open("SchizoModel.pkl","wb")
# joblib.dump(model,pkl_file)
# pkl_file.close()

In [50]:
thismodel = joblib.load("SchizoModel.pkl")
predict_tendency(thismodel, "i am feeling very depressed today")

(1, 15947)
The model predicts the statement to be: Schizophrenic


/home/godofgeeks/miniconda3/lib/python3.9/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
